Connected to quant-stack (Python 3.11.9)

In [ ]:
# QUANT SCIENCE LLC 
# THE QUANT SCIENTIST PRO ALGORITHMIC TRADER 
# LEVEL 2 PROGRAM
# ****
# CLINIC #6: INTRO TO QSRESEARCH

# QSResearch is a powerful library designed for quantitative research and strategy development.
# It provides tools for data preprocessing, feature engineering, backtesting, and performance analysis.

# * Goal:
#   - Learn how to use the QSResearch library to implement a quantitative research project.

# * Prerequisites:
#   - Clinic #5:
#     - You have already set up the QSConnect environment and connected to the database.
#     - You have created a Zipline bundle from the price data called `qspro_historical_prices_fmp`.

# * INSTALLATION INSTRUCTIONS ----
# - Install the `qsresearch` package from GitHub:
#   ```
#   pip install git+https://github.com/quant-science/QSResearch.git
#   ```

In [ ]:
# * LIBRARIES ----

import pandas as pd
import pytimetk as tk
import logging

from zipline.api import date_rules, time_rules

TODAY = "2025-07-31"

In [ ]:
# * 1.0 RECAP:

# 1. We have already set up the QSConnect environment and connected to the database in the previous clinic and we have created a Zipline bundle from the price data called `qspro_historical_prices_fmp` (Clinic #5).
# 2. You learned how to backtest a strategy with Zipline in Clinic #2. (Ref. 02_momentum_omega.ipynb)

# 3. The QSResearch library builds on top of Zipline for much faster algorithmic trading strategy research and development: https://github.com/quant-science/QSResearch/tree/master/qsresearch

In [ ]:
# * 2.0 QSRESEARCH ----

# * How it works:
# - QSResearch provides a structured way to define and run quantitative research projects.
# - It includes modules for data preprocessing, feature engineering, backtesting, and performance analysis.
# - And, most importantly, it provides an easy way to go from experiment design to backtesting and performance analysis that follows a 100% reproducible data science workflow.

In [ ]:
# * Example of a typical algo trading development workflow:
# 1. Extract Zipline Data from Zipline Bundle:
#    - The `get_zipline_history` function retrieves historical price data from the Zipline bundle.
#    - This data can be used for testing your functions.

from qsresearch.utils.zipline import get_zipline_history

# Make sure to adjust your bundle name. Symbols must be in the bundle.
# Here we are using the `qspro_historical_prices_fmp` bundle created in Clinic #5.

df = get_zipline_history(
    bundle_name="qspro_demo_historical_prices_fmp",
    symbols=["AAPL", "AMZN","PLTR"], # This is a small universe for testing
    end_date=TODAY,
    bar_count=252*3,  # 3 years of daily data
    frequency="1d"
)

df

field,date,symbol,close,high,low,open,volume
0,2022-07-28,AAPL,157.35,157.64,154.41,156.98,81378731.0
1,2022-07-28,AMZN,122.28,122.84,118.08,121.57,82245500.0
2,2022-07-28,PLTR,10.13,10.20,9.74,10.03,27172854.0
3,2022-07-29,AAPL,162.51,163.63,159.50,161.24,101786900.0
4,2022-07-29,AMZN,134.95,137.65,132.41,134.90,148892900.0
...,...,...,...,...,...,...,...
2263,2025-07-30,AMZN,230.19,231.80,229.29,231.64,32993300.0
2264,2025-07-30,PLTR,158.61,159.38,156.56,157.37,40261700.0
2265,2025-07-31,AAPL,207.57,209.84,207.16,208.49,64700006.0
2266,2025-07-31,AMZN,234.11,236.53,231.40,235.77,93411639.0


In [ ]:
# 2. Select Universe of Stocks:
#    - The `universe_screener` allows you to filter and select a universe

from qsresearch.preprocessors import universe_screener

df_screened = universe_screener(
    df,
    volume_top_n=3,
)

df_screened

2025-08-07 22:01:20,520 - INFO - Starting universe screening process.
2025-08-07 22:01:20,520 - INFO - Using max date from data: 2025-07-31 00:00:00
2025-08-07 22:01:20,520 - INFO - Lookback period: 2023-08-01 00:00:00 to 2025-07-31 00:00:00 (730 days)
2025-08-07 22:01:20,531 - INFO - Converted Pandas DataFrame to Polars DataFrame
2025-08-07 22:01:20,533 - INFO - Cast date column to Date type
2025-08-07 22:01:20,534 - INFO - Filtered data to time frame: 2023-08-01 00:00:00 to 2025-07-31 00:00:00, rows=1509
2025-08-07 22:01:20,534 - INFO - Skipped abnormal price movement filter (percent change filter).
2025-08-07 22:01:20,534 - INFO - Skipped volatility filter.
2025-08-07 22:01:20,534 - INFO - Skipped minimum average volume filter.
2025-08-07 22:01:20,537 - INFO - Applied volume filter, kept top 3 symbols, rows=1509
2025-08-07 22:01:20,537 - INFO - Skipped minimum last price filter.
2025-08-07 22:01:20,538 - INFO - Skipped minimum average price filter.
2025-08-07 22:01:20,539 - INFO - A

field,date,symbol,close,high,low,open,volume
0,2022-07-28,AAPL,157.35,157.64,154.41,156.98,81378731.0
1,2022-07-28,AMZN,122.28,122.84,118.08,121.57,82245500.0
2,2022-07-28,PLTR,10.13,10.20,9.74,10.03,27172854.0
3,2022-07-29,AAPL,162.51,163.63,159.50,161.24,101786900.0
4,2022-07-29,AMZN,134.95,137.65,132.41,134.90,148892900.0
...,...,...,...,...,...,...,...
2263,2025-07-30,AMZN,230.19,231.80,229.29,231.64,32993300.0
2264,2025-07-30,PLTR,158.61,159.38,156.56,157.37,40261700.0
2265,2025-07-31,AAPL,207.57,209.84,207.16,208.49,64700006.0
2266,2025-07-31,AMZN,234.11,236.53,231.40,235.77,93411639.0


In [ ]:
# 3. Preprocessing Price Data:
#    - The `preprocess_price_data` function cleans and prepares the price data for analysis.

from qsresearch.preprocessors import preprocess_price_data

df_preprocessed = preprocess_price_data(
    df_screened,
)

df_preprocessed.glimpse()

2025-08-07 22:01:20,568 - INFO - Preprocessed data shape: (2268, 17)


<class 'pandas.core.frame.DataFrame'>: 2268 rows of 17 columns
symbol:                  object            ['AAPL', 'AAPL', 'AAPL', 'AAP ...
date:                    datetime64[ms]    [Timestamp('2022-07-28 00:00: ...
open:                    float64           [156.98, 161.24, 161.01, 160. ...
high:                    float64           [157.64, 163.63, 163.59, 162. ...
low:                     float64           [154.41, 159.5, 160.89, 159.6 ...
close:                   float64           [157.35, 162.51, 161.51, 160. ...
volume:                  float64           [81378731.0, 101786900.0, 678 ...
flag_missing_original:   bool              [False, False, False, False,  ...
flag_zero_replaced:      bool              [False, False, False, False,  ...
flag_duplicate_dropped:  bool              [False, False, False, False,  ...
flag_large_gap:          bool              [False, False, False, False,  ...
flag_ohlc_corrected:     bool              [False, False, False, False,  ...
flag_high_pri

In [ ]:
# 4. Feature Engineering:
#    - The `add_technical_indicators` function adds features to the dataset. More feature engineering functions can be found in the `qsresearch.features` module.

from qsresearch.features import add_technical_indicators

df_engineered = add_technical_indicators(
    df_preprocessed,
    compute_rolling_risk=False,  # Takes long to run
    compute_qs_momentum=True,
    
)

df_engineered.glimpse()

2025-08-07 22:01:20,581 - INFO - Starting technical indicator augmentation.
2025-08-07 22:01:20,584 - INFO - Added cumulative return for 20 days
2025-08-07 22:01:20,585 - INFO - Added cumulative return for 60 days
2025-08-07 22:01:20,585 - INFO - Added cumulative return for 120 days
2025-08-07 22:01:20,585 - INFO - Added volatility for 20 days
2025-08-07 22:01:20,586 - INFO - Added volatility for 60 days
2025-08-07 22:01:20,586 - INFO - Added ATR for 14 days
2025-08-07 22:01:20,587 - INFO - Added volume ratio for 20 days
2025-08-07 22:01:20,587 - INFO - Added OBV
2025-08-07 22:01:20,593 - INFO - Added MACD with periods 12_26_9
2025-08-07 22:01:20,599 - INFO - Added MACD with periods 50_200_30
2025-08-07 22:01:20,603 - INFO - Added MACD histogram difference
2025-08-07 22:01:20,609 - INFO - Added PPO with periods 12_26
2025-08-07 22:01:20,615 - INFO - Added PPO with periods 50_200
2025-08-07 22:01:20,672 - INFO - Added QS Momentum
2025-08-07 22:01:20,681 - INFO - Added ROC features
2025-

<class 'pandas.core.frame.DataFrame'>: 2268 rows of 93 columns
symbol:                               object            ['AAPL', 'AAPL', ...
date:                                 datetime64[ms]    [Timestamp('2022 ...
open:                                 float64           [156.98, 161.24, ...
high:                                 float64           [157.64, 163.63, ...
low:                                  float64           [154.41, 159.5,  ...
close:                                float64           [157.35, 162.51, ...
volume:                               float64           [81378731.0, 101 ...
flag_missing_original:                bool              [False, False, F ...
flag_zero_replaced:                   bool              [False, False, F ...
flag_duplicate_dropped:               bool              [False, False, F ...
flag_large_gap:                       bool              [False, False, F ...
flag_ohlc_corrected:                  bool              [False, False, F ...
flag_high_pri

In [ ]:
# 5. Algorithms
#    - The `use_factor_as_signal` function allows you to use a specific factor (column) as a trading signal.

from qsresearch.strategies.factor.algorithms import use_factor_as_signal

train_data = df_engineered[df_engineered["date"] < TODAY]

predict_data = df_engineered[df_engineered["date"] == TODAY]

predict_data.glimpse()

factor_signal = use_factor_as_signal(
    train_data=train_data,
    predict_data=predict_data,
    factor_column="close_fastqsmom_21_252_126",
)

factor_signal

predict_data["factor_signal"] = factor_signal

predict_data.glimpse()

<class 'pandas.core.frame.DataFrame'>: 3 rows of 93 columns
symbol:                               object            ['AAPL', 'AMZN', ...
date:                                 datetime64[ms]    [Timestamp('2025 ...
open:                                 float64           [208.49, 235.77, ...
high:                                 float64           [209.84, 236.53, ...
low:                                  float64           [207.16, 231.4,  ...
close:                                float64           [207.57, 234.11, ...
volume:                               float64           [64700006.0, 934 ...
flag_missing_original:                bool              [False, False, F ...
flag_zero_replaced:                   bool              [False, False, F ...
flag_duplicate_dropped:               bool              [False, False, F ...
flag_large_gap:                       bool              [False, False, F ...
flag_ohlc_corrected:                  bool              [False, False, F ...
flag_high_price:

In [ ]:
# 6. Portfolio Construction:
#    - The `long_short_equal_weight_portfolio` function constructs a portfolio based on the factor signal.

from qsresearch.strategies.factor.portfolio_construction import long_short_equal_weight_portfolio

weights = long_short_equal_weight_portfolio(
    predictions=predict_data["factor_signal"],
    num_long_positions=2,
    long_threshold=0,
)

weights

{2267: 0.5, 1511: 0.5}

In [ ]:
# 7. Backtesting:
#   - The `run_backtest` function allows you to backtest the portfolio using historical data running this algorithm in a loop using Zipline.
#   - We develop a configuration dictionary that defines the backtest parameters and the algorithm to run.
#   - The CONFIG integrates our workflow steps into a single backtest configuration.

from qsresearch.strategies.factor import run_backtest

PREDICTOR_COLS = ["close_fastqsmom_21_252_126"]

CONFIG = {
    
    # MLFlow Tracking
    # Must include:
    # start_date: Start date of the backtest (pd.Timestamp).
    # end_date: End date of the backtest (pd.Timestamp).
    # capital_base: Initial capital for the backtest (float).
    # bundle_name: Name of the Zipline data bundle (str).
    
    # Optional keys:
    # calendar_name: Trading calendar name (default: 'NYSE').
    # custom_handle_data: Custom handle_data function (default: default_handle_data).
    # mlflow_tracking_uri:  MLflow tracking server URI (e.g., 'http://mlflow-server:5000').
    #                       An empty string, or a local file path, prefixed with file:/ 
    #                       Data is stored locally at the provided file (or ./mlruns if empty)
    #                       Can be an HTTP URI like https://my-tracking-server:5000 (or :8301)
    # mlflow_tracking_port: MLflow tracking server port (default: None) - don't use both port and URI.
    # mlflow_artifact_root: Artifact storage location (e.g., 's3://my-bucket/mlflow/artifacts').
    # mlflow_nested_run: Whether to create a nested run (default: False).
    # mlflow_log_metrics_frequency: Frequency for logging metrics (e.g., 'daily', default: None).
    # mlflow_artifact_subdir: Subdirectory for artifacts (e.g., 'momentum_backtest'). """
    
    "use_mlflow": True,
    "mlflow_tracking_uri": "/Users/brucebrownlee/dev/github/Resident/QS-Project/Clinic-06/mlruns",  # Local file storage for testing
    "mlflow_experiment_name": "Test Strategies",
    "mlflow_run_name": "Test Strategy 5",
    "mlflow_tags": {
        "strategy": "test", "portfolio": "equal_weight"
    },

    # BACKTEST PARAMETERS:
    "bundle_name": "qspro_demo_historical_prices_fmp",
    "start_date": pd.Timestamp("2025-05-01"), # NOTE - I'm making this small for testing
    "end_date": pd.Timestamp(TODAY),
    "capital_base": 1_000_000,    
    "benchmark_symbol": "SPY",  # Set to None to skip benchmark
    "window_length": 252 * 3,  # zipline bar count window for training and prediction
    "frequency": "1d",
    "predictor_cols": PREDICTOR_COLS,
    "calendar_name": "NYSE",  # Default calendar, can be changed
    "extra_init": None,  # Optional custom initialization function
    "custom_handle_data": None,  # Optional custom handle_data function
    
    # BACKTEST FUNCTIONS
    "rebalance_schedule": {
        "date_rule": date_rules.month_start(),
        "time_rule": time_rules.market_open(minutes=60),
    },
    "transaction_costs": {
        "slippage": {"spread": 0.01},
        "commission": {"cost": 0.005, "min_trade_cost": 0},
    },
    # # Add stop-loss settings
    # 'stop_loss': {
    #     'long_threshold': 0.10,  # 10% stop-loss
    #     'short_threshold': 0.10,  # 10% stop-loss
    #     'date_rule': date_rules.every_day(),
    #     'time_rule': time_rules.market_open(minutes=60),
    # },
    
    # Preprocessing steps applied sequentially
    "preprocess": [
        {
            "name": "screener",
            "func": universe_screener,  # Assuming this is from your module
            "params": {
                "lookback_days": 2 * 365,
                "volume_top_n": 10, # NOTE - I'M MAKING THIS VERY SMALL FOR TESTING
                "momentum_top_n": None,
                "percent_change_filter": False,
                "max_percent_change": 0.35,
                "volatility_filter": True,
                "max_volatility": 0.25,
                "min_avg_volume": 100_000,
                "min_avg_price": 4.0,
                "min_last_price": 5.0,
                "symbol_column": "symbol",
                "date_column": "date",
                "close_column": "close",
                "volume_column": "volume",
            },
        },
        {
            "name": "price_preprocessor",
            "func": preprocess_price_data,
            "params": {
                "min_trading_days": 252 * 2,
                "remove_low_trading_days": True,
                "remove_large_gaps": True,
                "remove_low_volume": True,
                "symbol_column": "symbol",
                "date_column": "date",
                "open_column": "open",
                "high_column": "high",
                "low_column": "low",
                "close_column": "close",
                "volume_column": "volume",
                "engine": "polars",
            },
        },
        {
            "name": "technical_indicators",
            "func": add_technical_indicators,
            "params": {
                "date_column": "date",
                "symbol_column": "symbol",
                "close_column": "close",
                "high_column": "high",
                "low_column": "low",
                "volume_column": "volume",
                "compute_rolling_risk": False,
                "compute_qs_momentum": True,  # Add QS Momentum Factor
            },
        }
    ],
    
    # Algorithm
    "algorithm": {
        "func": use_factor_as_signal,
        "params": {
            "factor_column": PREDICTOR_COLS[0],
        },
    },
    
    # Portfolio Construction:
    "portfolio_strategy": {
        "func": long_short_equal_weight_portfolio,
        "params": {
            "num_long_positions": 5, # NOTE - I'M MAKING THIS SMALL FOR TESTING
            "long_threshold": 1.00,
            # 'num_short_positions': 20,
            # 'short_threshold': -1.00,
        },
    },
}

performance_df = run_backtest(CONFIG)

performance_df.glimpse()

2025-08-07 22:01:21,647 - INFO - Initializing backtest
2025-08-07 22:01:21,681 - INFO - Benchmark symbol: SPY
2025-08-07 22:01:21,779 - INFO - No 'stop_loss' key found in config; stop-loss disabled.
2025-08-07 22:01:21,823 - INFO - Handling data for 2025-05-01 20:00:00+00:00
2025-08-07 22:01:21,825 - INFO - Rebalancing on 2025-05-01 00:00:00
2025-08-07 22:01:30,221 - INFO - Applying preprocessing step: screener
2025-08-07 22:01:30,222 - INFO - Starting universe screening process.
2025-08-07 22:01:30,223 - INFO - Using max date from data: 2025-05-01 00:00:00
2025-08-07 22:01:30,223 - INFO - Lookback period: 2023-05-02 00:00:00 to 2025-05-01 00:00:00 (730 days)
2025-08-07 22:01:30,321 - INFO - Converted Pandas DataFrame to Polars DataFrame
2025-08-07 22:01:30,326 - INFO - Cast date column to Date type
2025-08-07 22:01:30,331 - INFO - Filtered data to time frame: 2023-05-02 00:00:00 to 2025-05-01 00:00:00, rows=1207703
2025-08-07 22:01:30,331 - INFO - Skipped abnormal price movement filte

Generating tear sheet...


Start date,2025-05-01
End date,2025-07-31
Total months,3
,Backtest
Annual return,168.386%
Cumulative returns,27.994%
Annual volatility,56.327%
Sharpe ratio,2.04
Calmar ratio,12.68
Stability,0.83
Max drawdown,-13.278%


Worst drawdown periods,Net drawdown in %,Peak date,Valley date,Recovery date,Duration
0,13.28,2025-05-01,2025-05-06,2025-05-13,9
1,9.95,2025-06-03,2025-06-05,2025-06-11,7
2,9.40,2025-06-26,2025-07-01,2025-07-14,13
3,7.86,2025-05-14,2025-05-21,2025-05-30,13
4,3.20,2025-07-17,2025-07-22,2025-07-23,5


2025-08-07 22:01:44,253 - INFO - Using categorical units to plot a list of strings that are all parsable as floats or dates. If these strings should be plotted as numbers, cast to the appropriate data type before plotting.
2025-08-07 22:01:44,255 - INFO - Using categorical units to plot a list of strings that are all parsable as floats or dates. If these strings should be plotted as numbers, cast to the appropriate data type before plotting.


...

,5D
Mean Factor Rank Autocorrelation,0.984


2025-08-07 22:01:47,165 - INFO - Logging Alphalens artifact to: performance_metrics


HTML tear sheet saved as: alphalens_tear_sheet.html


2025-08-07 22:01:47,386 - INFO - MLflow run URL: /Users/brucebrownlee/dev/github/Resident/QS-Project/Clinic-06/mlruns/617749832001754610/5914a17221814b769e08e7ff9caf522e/artifacts
2025-08-07 22:01:47,387 - INFO - Use the MLflow UI to view the run details by running this command in terminal: `mlflow ui`


<class 'pandas.core.frame.DataFrame'>: 63 rows of 40 columns
period_open:              datetime64[ns, UTC] [Timestamp('2025-05-01 13: ...
period_close:             datetime64[ns, UTC] [Timestamp('2025-05-01 20: ...
shorts_count:             int64             [0, 0, 0, 0, 0, 0, 0, 0, 0,  ...
longs_count:              int64             [0, 1, 1, 1, 1, 1, 1, 1, 1,  ...
short_value:              float64           [0.0, 0.0, 0.0, 0.0, 0.0, 0. ...
pnl:                      float64           [0.0, -86.05000000016298, -4 ...
starting_value:           float64           [0.0, 0.0, 1069429.4, 106504 ...
capital_used:             float64           [0.0, -1069515.45, 0.0, 0.0, ...
returns:                  float64           [0.0, -8.605000000017071e-05 ...
ending_exposure:          float64           [0.0, 1069429.4, 1065040.849 ...
long_exposure:            float64           [0.0, 1069429.4, 1065040.849 ...
short_exposure:           float64           [0.0, 0.0, 0.0, 0.0, 0.0, 0. ...
positions:     

In [ ]:
# * MLFLOW TRACKING ----

# Run in terminal:
#   mlflow server

# * NEXT STEPS:

# - Knowledge Check: Try adding a new function that adds a custom feature to the dataset.

# - Now you know how it works, we'll examine the QS Momentum Factor strategy.

In [ ]:
performance_df

,period_open,period_close,shorts_count,longs_count,short_value,pnl,starting_value,capital_used,returns,ending_exposure,...,excess_return,treasury_period_return,benchmark_period_return,benchmark_volatility,max_drawdown,trading_days,algo_volatility,period_label,max_leverage,sharpe
2025-05-01 00:00:00+00:00,2025-05-01 13:31:00+00:00,2025-05-01 20:00:00+00:00,0,0,0.0,0.00,0.00,0.00,0.000000,0.00,...,0.0,0.0,0.007087,NaN,0.000000,1,NaN,2025-05,0.000000,NaN
2025-05-02 00:00:00+00:00,2025-05-02 13:31:00+00:00,2025-05-02 20:00:00+00:00,0,1,0.0,-86.05,0.00,-1069515.45,-0.000086,1069429.40,...,0.0,0.0,0.022036,0.087074,-0.000086,2,0.000966,2025-05,1.069521,-11.224972
2025-05-05 00:00:00+00:00,2025-05-05 13:31:00+00:00,2025-05-05 20:00:00+00:00,0,1,0.0,-4388.55,1069429.40,0.00,-0.004389,1065040.85,...,0.0,0.0,0.016176,0.164977,-0.004475,3,0.039837,2025-05,1.069828,-9.435973
2025-05-06 00:00:00+00:00,2025-05-06 13:31:00+00:00,2025-05-06 20:00:00+00:00,0,1,0.0,-128300.55,1065040.85,0.00,-0.128877,936740.30,...,0.0,0.0,0.007682,0.173402,-0.132775,4,1.011615,2025-05,1.080159,-8.304732
2025-05-07 00:00:00+00:00,2025-05-07 13:31:00+00:00,2025-05-07 20:00:00+00:00,0,1,0.0,13940.10,936740.30,0.00,0.016074,950680.40,...,0.0,0.0,0.011920,0.151015,-0.132775,5,0.943705,2025-05,1.080159,-6.263400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-07-25 00:00:00+00:00,2025-07-25 13:31:00+00:00,2025-07-25 20:00:00+00:00,0,1,0.0,31847.02,1251733.38,0.00,0.025442,1283580.40,...,0.0,0.0,0.148880,0.121330,-0.132775,59,0.580538,2025-07,1.080159,2.131068
2025-07-28 00:00:00+00:00,2025-07-28 13:31:00+00:00,2025-07-28 20:00:00+00:00,0,1,0.0,-7436.36,1283580.40,0.00,-0.005793,1276144.04,...,0.0,0.0,0.148592,0.120419,-0.132775,60,0.576015,2025-07,1.080159,2.069762
2025-07-29 00:00:00+00:00,2025-07-29 13:31:00+00:00,2025-07-29 20:00:00+00:00,0,1,0.0,-13256.12,1276144.04,0.00,-0.010388,1262887.92,...,0.0,0.0,0.145562,0.119839,-0.132775,61,0.572021,2025-07,1.080159,1.975028
2025-07-30 00:00:00+00:00,2025-07-30 13:31:00+00:00,2025-07-30 20:00:00+00:00,0,1,0.0,19156.71,1262887.92,0.00,0.015169,1282044.63,...,0.0,0.0,0.144119,0.119064,-0.132775,62,0.567722,2025-07,1.080159,2.066487
